In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gmaps
import gmaps.datasets
import ipyleaflet
from IPython.display import display

In [3]:
#importing environment variables
from dotenv import dotenv_values
env_variables = dotenv_values('Ignore.env')

In [4]:
df = pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Intermediate Data/Wrangled_data')

/var/folders/7b/nyzqb3pd1zn309ndm96_lqcm0000gn/T/ipykernel_21097/3929973976.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Intermediate Data/Wrangled_data')


In [5]:
df.head()

,Unnamed: 0,IDCyc,WhenStartPickup,IDMemb,WhenReg,Addr1,Addr2,City,St,Zip,...,full_address,latitude,longitude,Pickup_lat,Pickup_long,Distance_to_pickup,Years_member,quarter,month,holiday
0,0,334,2023-05-18 15:45:00,1015,2008-09-09,9390 Lakewood Cir,NaN,Norwalk,IA,50211,...,"9390 Lakewood Cir,Norwalk,IA,50211",41.497804,-93.687343,41.611449,-93.686915,7.842931,14.697688,2023Q2,5,0
1,1,333,2023-05-04 15:45:00,1015,2008-09-09,9390 Lakewood Cir,NaN,Norwalk,IA,50211,...,"9390 Lakewood Cir,Norwalk,IA,50211",41.497804,-93.687343,41.611449,-93.686915,7.842931,14.659332,2023Q2,5,0
2,2,332,2023-04-20 15:45:00,1015,2008-09-09,9390 Lakewood Cir,NaN,Norwalk,IA,50211,...,"9390 Lakewood Cir,Norwalk,IA,50211",41.497804,-93.687343,41.611449,-93.686915,7.842931,14.620976,2023Q2,4,0
3,3,331,2023-04-06 15:45:00,1015,2008-09-09,9390 Lakewood Cir,NaN,Norwalk,IA,50211,...,"9390 Lakewood Cir,Norwalk,IA,50211",41.497804,-93.687343,41.611449,-93.686915,7.842931,14.582620,2023Q2,4,0
4,4,330,2023-03-23 15:45:00,1015,2008-09-09,9390 Lakewood Cir,NaN,Norwalk,IA,50211,...,"9390 Lakewood Cir,Norwalk,IA,50211",41.497804,-93.687343,41.611449,-93.686915,7.842931,14.544264,2023Q1,3,0


In [6]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)


In [7]:
df.shape

(268015, 49)

In [8]:
df.columns

Index(['IDCyc', 'WhenStartPickup', 'IDMemb', 'WhenReg', 'Addr1', 'Addr2',
       'City', 'St', 'Zip', 'CDLocLast', 'NumLogins', 'CkFounder', 'HowHear',
       'CDRegMemb', 'SaleNom', 'Baked Goods', 'Beverages',
       'Classes and Events', 'Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center',
       'full_address', 'latitude', 'longitude', 'Pickup_lat', 'Pickup_long',
       'Distance_to_pickup', 'Years_member', 'quarter', 'month', 'holiday'],
      dtype='object')

In [9]:
#Filling in all NAs from the different sales categories with 0's
df[['SaleNom', 'Baked Goods', 'Beverages', 'Classes and Events',
       'Condiments + Sauces', 'Dairy', 'Dried Herbs + Spices', 'Eggs',
       'Grains, Flours, Cereal + Pastas', 'Handmade Home Goods + Gifts',
       'Honey, Syrups, Jams + Jellies', 'Iowa Food Co-op Shop',
       'Local Produce', 'Meat - Beef', 'Meat - Chicken + Capon', 'Meat - Pork',
       'Meats - Other', 'Non-Food Items', 'Nuts', 'Other Protein Sources',
       'Personal Care', 'Pet + Animal Care', 'Prepared Foods', 'Snacks',
       'The Garden Center']]=df[['SaleNom', 'Baked Goods', 'Beverages', 'Classes and Events',
       'Condiments + Sauces', 'Dairy', 'Dried Herbs + Spices', 'Eggs',
       'Grains, Flours, Cereal + Pastas', 'Handmade Home Goods + Gifts',
       'Honey, Syrups, Jams + Jellies', 'Iowa Food Co-op Shop',
       'Local Produce', 'Meat - Beef', 'Meat - Chicken + Capon', 'Meat - Pork',
       'Meats - Other', 'Non-Food Items', 'Nuts', 'Other Protein Sources',
       'Personal Care', 'Pet + Animal Care', 'Prepared Foods', 'Snacks',
       'The Garden Center']].fillna(0)

In [10]:
cleaning_df = df[['IDCyc','IDMemb','Baked Goods', 'Beverages',
       'Classes and Events', 'Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center','SaleNom']]

In [11]:
#Need to sort df oldest to newest first as we start making columns that cumulate over time.
cleaning_df = cleaning_df.sort_values(by=['IDMemb','IDCyc'], ascending=True)

In [12]:
cleaning_df.head()

,IDCyc,IDMemb,Baked Goods,Beverages,Classes and Events,Condiments + Sauces,Dairy,Dried Herbs + Spices,Eggs,"Grains, Flours, Cereal + Pastas",...,Meats - Other,Non-Food Items,Nuts,Other Protein Sources,Personal Care,Pet + Animal Care,Prepared Foods,Snacks,The Garden Center,SaleNom
49,285,1015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,286,1015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,287,1015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,288,1015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,289,1015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
cleaning_df[cleaning_df['IDMemb']==1078]

,IDCyc,IDMemb,Baked Goods,Beverages,Classes and Events,Condiments + Sauces,Dairy,Dried Herbs + Spices,Eggs,"Grains, Flours, Cereal + Pastas",...,Meats - Other,Non-Food Items,Nuts,Other Protein Sources,Personal Care,Pet + Animal Care,Prepared Foods,Snacks,The Garden Center,SaleNom
2649,285,1078,6.0,0.0,0.0,24.00,6.333333,0.0,4.700,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,4.500000,0.00,0.00,145.50
2648,286,1078,0.0,16.0,0.0,0.00,0.000000,0.0,8.500,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,8.00,0.00,114.29
2647,287,1078,0.0,0.0,0.0,24.00,5.750000,0.0,4.250,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,8.500000,8.00,0.00,94.50
2646,288,1078,0.0,16.0,0.0,0.00,0.000000,0.0,8.500,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,108.14
2645,289,1078,0.0,0.0,0.0,0.00,13.000000,0.0,3.750,0.0,...,0.000000,0.0,0.0,0.0,4.250000,0.0,0.000000,0.00,0.00,74.45
2644,290,1078,0.0,0.0,0.0,0.00,7.250000,0.0,7.500,0.0,...,12.520000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,55.27
2643,291,1078,6.0,16.0,0.0,0.00,3.583333,0.0,7.500,0.0,...,10.300000,0.0,0.0,0.0,0.000000,0.0,18.500000,4.50,0.00,177.86
2642,292,1078,0.0,0.0,0.0,0.00,0.000000,0.0,0.000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.00
2641,293,1078,0.0,0.0,0.0,0.00,0.000000,0.0,7.500,9.5,...,0.000000,0.0,0.0,0.0,0.000000,0.0,16.125000,4.50,0.00,128.16
2640,294,1078,6.0,0.0,0.0,7.00,0.000000,0.0,8.000,0.0,...,24.320000,0.0,0.0,0.0,4.000000,0.0,20.000000,5.00,0.00,142.22


In [14]:
#Creating a Cumulative Sum column to add customer orders as they go since we can't use total sales in our final model.
cleaning_df["Cumulative_Sum"] = cleaning_df.groupby("IDMemb")["SaleNom"].transform(lambda x: x.shift().fillna(0).cumsum())

In [15]:
#Column of 1s and 0s for whether or not a person ordered during that ordering period
cleaning_df['Ordered'] = cleaning_df['SaleNom'].apply((lambda x: 1 if x>0 else 0))

In [16]:
#Summing the new column to have a cumulative sum of total orders for each member.
cleaning_df['Cumulative_Number_of_Orders'] = cleaning_df.groupby("IDMemb")["Ordered"].transform(lambda x: x.shift().fillna(0).cumsum())

In [17]:
#Calculating average order when the person orders. 
cleaning_df['order_per_cycle_when_ordering'] = cleaning_df['Cumulative_Sum']/cleaning_df['Cumulative_Number_of_Orders']

In [18]:
column_list = ['Baked Goods', 'Beverages','Classes and Events','Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center']

for column in column_list:
    cleaning_df[f'{column}_Cum'] = cleaning_df.groupby("IDMemb")[column].transform(lambda x: x.shift().fillna(0).cumsum())
    cleaning_df.drop(column, axis=1, inplace=True)

In [19]:
cleaning_df.head()

,IDCyc,IDMemb,SaleNom,Cumulative_Sum,Ordered,Cumulative_Number_of_Orders,order_per_cycle_when_ordering,Baked Goods_Cum,Beverages_Cum,Classes and Events_Cum,...,Meat - Pork_Cum,Meats - Other_Cum,Non-Food Items_Cum,Nuts_Cum,Other Protein Sources_Cum,Personal Care_Cum,Pet + Animal Care_Cum,Prepared Foods_Cum,Snacks_Cum,The Garden Center_Cum
49,285,1015,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,286,1015,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,287,1015,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,288,1015,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,289,1015,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
cleaning_df[cleaning_df['IDMemb']==1016]

,IDCyc,IDMemb,SaleNom,Cumulative_Sum,Ordered,Cumulative_Number_of_Orders,order_per_cycle_when_ordering,Baked Goods_Cum,Beverages_Cum,Classes and Events_Cum,...,Meat - Pork_Cum,Meats - Other_Cum,Non-Food Items_Cum,Nuts_Cum,Other Protein Sources_Cum,Personal Care_Cum,Pet + Animal Care_Cum,Prepared Foods_Cum,Snacks_Cum,The Garden Center_Cum
99,285,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,286,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,287,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,288,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,289,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,290,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,291,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,292,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,293,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,294,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
cleaning_df.sort_values(by=['IDCyc','IDMemb'], ascending=True,inplace = True)

In [22]:
#Cumulative count of how many cycles this person has been a member.
cleaning_df['Cycles_as_member'] = cleaning_df.groupby('IDMemb')['IDMemb'].cumcount() + 1

In [23]:
cleaning_df.head()

,IDCyc,IDMemb,SaleNom,Cumulative_Sum,Ordered,Cumulative_Number_of_Orders,order_per_cycle_when_ordering,Baked Goods_Cum,Beverages_Cum,Classes and Events_Cum,...,Meats - Other_Cum,Non-Food Items_Cum,Nuts_Cum,Other Protein Sources_Cum,Personal Care_Cum,Pet + Animal Care_Cum,Prepared Foods_Cum,Snacks_Cum,The Garden Center_Cum,Cycles_as_member
49,285,1015,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
99,285,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
149,285,1017,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
199,285,1018,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
249,285,1019,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [24]:
#Calulate how much each customer orders per cycle by dividing their total orders by the number of cycles they have been a member.
cleaning_df['order_per_cycle'] = cleaning_df['Cumulative_Sum']/(cleaning_df['Cycles_as_member']-1)

In [25]:
cleaning_df.head()

,IDCyc,IDMemb,SaleNom,Cumulative_Sum,Ordered,Cumulative_Number_of_Orders,order_per_cycle_when_ordering,Baked Goods_Cum,Beverages_Cum,Classes and Events_Cum,...,Non-Food Items_Cum,Nuts_Cum,Other Protein Sources_Cum,Personal Care_Cum,Pet + Animal Care_Cum,Prepared Foods_Cum,Snacks_Cum,The Garden Center_Cum,Cycles_as_member,order_per_cycle
49,285,1015,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
99,285,1016,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
149,285,1017,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
199,285,1018,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
249,285,1019,0.0,0.0,0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN


In [26]:
cleaning_df['order_per_cycle'] = cleaning_df['order_per_cycle'].fillna(0)

In [27]:
cleaning_df['order_per_cycle_when_ordering'] = cleaning_df['order_per_cycle_when_ordering'].fillna(0)

In [28]:
cleaning_df.head()

,IDCyc,IDMemb,SaleNom,Cumulative_Sum,Ordered,Cumulative_Number_of_Orders,order_per_cycle_when_ordering,Baked Goods_Cum,Beverages_Cum,Classes and Events_Cum,...,Non-Food Items_Cum,Nuts_Cum,Other Protein Sources_Cum,Personal Care_Cum,Pet + Animal Care_Cum,Prepared Foods_Cum,Snacks_Cum,The Garden Center_Cum,Cycles_as_member,order_per_cycle
49,285,1015,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
99,285,1016,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
149,285,1017,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
199,285,1018,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
249,285,1019,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0


In [29]:
cleaning_df.columns

Index(['IDCyc', 'IDMemb', 'SaleNom', 'Cumulative_Sum', 'Ordered',
       'Cumulative_Number_of_Orders', 'order_per_cycle_when_ordering',
       'Baked Goods_Cum', 'Beverages_Cum', 'Classes and Events_Cum',
       'Condiments + Sauces_Cum', 'Dairy_Cum', 'Dried Herbs + Spices_Cum',
       'Eggs_Cum', 'Grains, Flours, Cereal + Pastas_Cum',
       'Handmade Home Goods + Gifts_Cum', 'Honey, Syrups, Jams + Jellies_Cum',
       'Iowa Food Co-op Shop_Cum', 'Local Produce_Cum', 'Meat - Beef_Cum',
       'Meat - Chicken + Capon_Cum', 'Meat - Pork_Cum', 'Meats - Other_Cum',
       'Non-Food Items_Cum', 'Nuts_Cum', 'Other Protein Sources_Cum',
       'Personal Care_Cum', 'Pet + Animal Care_Cum', 'Prepared Foods_Cum',
       'Snacks_Cum', 'The Garden Center_Cum', 'Cycles_as_member',
       'order_per_cycle'],
      dtype='object')

In [30]:
df.columns

Index(['IDCyc', 'WhenStartPickup', 'IDMemb', 'WhenReg', 'Addr1', 'Addr2',
       'City', 'St', 'Zip', 'CDLocLast', 'NumLogins', 'CkFounder', 'HowHear',
       'CDRegMemb', 'SaleNom', 'Baked Goods', 'Beverages',
       'Classes and Events', 'Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center',
       'full_address', 'latitude', 'longitude', 'Pickup_lat', 'Pickup_long',
       'Distance_to_pickup', 'Years_member', 'quarter', 'month', 'holiday'],
      dtype='object')

In [31]:
df.drop(['Baked Goods', 'Beverages',
       'Classes and Events', 'Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center'],axis=1, inplace=True)

In [32]:
print(df.shape,cleaning_df.shape)

(268015, 25) (268015, 33)


In [33]:
df.columns

Index(['IDCyc', 'WhenStartPickup', 'IDMemb', 'WhenReg', 'Addr1', 'Addr2',
       'City', 'St', 'Zip', 'CDLocLast', 'NumLogins', 'CkFounder', 'HowHear',
       'CDRegMemb', 'SaleNom', 'full_address', 'latitude', 'longitude',
       'Pickup_lat', 'Pickup_long', 'Distance_to_pickup', 'Years_member',
       'quarter', 'month', 'holiday'],
      dtype='object')

In [34]:
df=df.merge(cleaning_df,how='left', on =['IDCyc','IDMemb','SaleNom'])

In [35]:
df.shape

(268015, 55)

In [36]:
df = df.sort_values(['IDMemb','IDCyc'],ascending = True)

In [37]:
filepath = '/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Intermediate Data/cleaned_data'
df.to_csv(filepath)